## 1. Import libraies

In [2]:
import torch
from torch.autograd.functional import jacobian

## 2. Jacobian matrix

$
\mathbf{a} = \begin{bmatrix}
a_{1} & a_{2} & a_{3}
\end{bmatrix}
,
\mathbf{x} = \begin{bmatrix}
x_{1} & x_{2} & x_{3}
\end{bmatrix}
,
func = \begin{bmatrix}
a_{1} * x_{1} & a_{2} * x_{2} & a_{3} * x_{3}
\end{bmatrix}
=
\begin{bmatrix}
y_{1} & y_{2} & y_{3}
\end{bmatrix}
$

In [3]:
a = torch.randn(3)
print('a =', a)
def func(x):
    return a * x

a = tensor([-0.6241,  0.5889, -1.4825])


$
\left.
J =
\left(
  \begin{array}{ccc}
  \frac{\partial y_1}{\partial x_1} & \frac{\partial y_1}{\partial x_2} & \frac{\partial y_1}{\partial x_3} \\
  \frac{\partial y_2}{\partial x_1} & \frac{\partial y_2}{\partial x_2} & \frac{\partial y_2}{\partial x_3} \\
  \frac{\partial y_3}{\partial x_1} & \frac{\partial y_3}{\partial x_2} & \frac{\partial y_3}{\partial x_3}
  \end{array}
  \right.
\right)
$

In [4]:
x = torch.randn(3)
print('\nx =', x)
J = jacobian(func, x)
print('\nJ =', J)


x = tensor([-0.2205, -0.3078, -0.8473])

J = tensor([[-0.6241,  0.0000, -0.0000],
        [-0.0000,  0.5889, -0.0000],
        [-0.0000,  0.0000, -1.4825]])


## 3. Calculated gradient

In [5]:
x = torch.randn(3, requires_grad=True)
print('x =', x)
y = func(x)
print('\ny =', y)

x = tensor([-0.9950,  0.1402, -0.3712], requires_grad=True)

y = tensor([0.6210, 0.0825, 0.5503], grad_fn=<MulBackward0>)


In [6]:
# solution 1: backward()
# If y is a scalar, just y.backward()
# If not, need torch.ones_like(y), which equivalent to ∂L/∂y (L is equivalent to the uppermost, and backward mode generally assumes that the derivative of L is 1)
y.backward(torch.ones_like(y))
x.grad

tensor([-0.6241,  0.5889, -1.4825])

In [7]:
# solution 2: v.T @ J
# Instead of computing the Jacobian matrix itself, PyTorch allows you to compute Jacobian Product v.T
torch.ones_like(y) @ jacobian(func, x)   # torch.ones_like(y) equal to v.T

tensor([-0.6241,  0.5889, -1.4825])

## 4. use ```torch.autograd.grad()``` to get $u_x, u_{xx}$

will use it in PINN

In [8]:
def func(x, y):
    return x.exp() + y   # x.exp() + y**2

a = torch.randn(3, 1, requires_grad=True)
b = torch.randn(3, 1, requires_grad=True)
u = func(a, b)

u_x = torch.autograd.grad(u, a, grad_outputs=torch.ones_like(u),
                          retain_graph=True, create_graph=True)[0]
print('u_x =', u_x)
u_y = torch.autograd.grad(u, b, grad_outputs=torch.ones_like(u),
                          retain_graph=True, create_graph=True)[0]
print('u_y =', u_y)

u_xx = torch.autograd.grad(u_x, a, grad_outputs=torch.ones_like(u_x),
                           retain_graph=True, create_graph=True)[0]
print('u_xx =', u_xx)

# u_yy = torch.autograd.grad(u_y, b, grad_outputs=torch.ones_like(u_y),
#                            retain_graph=True, create_graph=True)[0]
# print('u_yy =', u_yy)


u_x = tensor([[0.4811],
        [0.3443],
        [0.0684]], grad_fn=<MulBackward0>)
u_y = tensor([[1.],
        [1.],
        [1.]])
u_xx = tensor([[0.4811],
        [0.3443],
        [0.0684]], grad_fn=<MulBackward0>)


In [11]:
def func(x, y):
    return 0.5*x*y, 0.1*y

a = torch.randn(3, 1, requires_grad=True)
b = torch.randn(3, 1, requires_grad=True)
x,y = func(a, b)
print(x.type)
u_x = torch.autograd.grad(x, a, grad_outputs=torch.ones_like(u),
                          retain_graph=True, create_graph=True)[0]
print('u_x =', u_x)
u_y = torch.autograd.grad(y, b, grad_outputs=torch.ones_like(u),
                          retain_graph=True, create_graph=True)[0]
print('u_y =', u_y)

u_xx = torch.autograd.grad(u_x, a, grad_outputs=torch.ones_like(u_x),
                           retain_graph=True, create_graph=True)[0]
print('u_xx =', u_xx)

u_yy = torch.autograd.grad(u_y, b, grad_outputs=torch.ones_like(u_y),
                           retain_graph=True, create_graph=True)[0]
print('u_yy =', u_yy)

u_xy = torch.autograd.grad(u_x, b, grad_outputs=torch.ones_like(u_x),
                           retain_graph=True, create_graph=True)[0]
print('u_xy =', u_xy)

u_yx = torch.autograd.grad(u_y, a, grad_outputs=torch.ones_like(u_y),
                           retain_graph=True, create_graph=True)[0]
print('u_yx =', u_yx)

<built-in method type of Tensor object at 0x000002684DDD0E90>
u_x = tensor([[ 0.1886],
        [-0.1228],
        [ 0.2779]], grad_fn=<MulBackward0>)
u_y = tensor([[0.1000],
        [0.1000],
        [0.1000]])


RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.